In [5]:
import boto3
from datetime import date
import time
import uuid

## Connect to DynamoDB

Initially, we connect to our local database we started as "dynamodb" container:

In [6]:
client = boto3.client('dynamodb', 
  endpoint_url='http://dynamo:8000', 
  region_name='eu-central-1',
  aws_access_key_id="an-access-key",
  aws_secret_access_key="a-secret-access-key"
)
client.list_tables()

ClientError: An error occurred (UnrecognizedClientException) when calling the ListTables operation: The Access Key ID or security token is invalid.

## Create Table

Let's setup the table with our attributes and indexes:

In [16]:
def create_app_table():
    return client.create_table(
        TableName="chat",
        KeySchema=[
            {"AttributeName": "PK", "KeyType": "HASH"},
            {"AttributeName": "SK", "KeyType": "RANGE"},
        ],
        AttributeDefinitions=[
            {"AttributeName": "PK", "AttributeType": "S"},
            {"AttributeName": "SK", "AttributeType": "S"},
            {"AttributeName": "GSI1PK", "AttributeType": "S"},
            {"AttributeName": "GSI1SK", "AttributeType": "S"},
        ],
        ProvisionedThroughput={"ReadCapacityUnits": 10, "WriteCapacityUnits": 10},
        GlobalSecondaryIndexes=[
            {
                "IndexName": "GSI1",
                "KeySchema": [
                    {"AttributeName": "GSI1PK", "KeyType": "HASH"},
                    {"AttributeName": "GSI1SK", "KeyType": "RANGE"},
                ],
                "Projection": {
                    "ProjectionType": "INCLUDE",
                    "NonKeyAttributes": ["ChanName", "JoinedAt"],
                },
                "ProvisionedThroughput": {
                    "ReadCapacityUnits": 10,
                    "WriteCapacityUnits": 10,
                },
            },
        ],
    )

In [4]:
def delete_app_table():
    try:
        return client.delete_table(TableName="chat")
    except:
        return False

In [5]:
delete_app_table()
create_app_table()

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'PK',
    'AttributeType': 'S'},
   {'AttributeName': 'SK', 'AttributeType': 'S'},
   {'AttributeName': 'GSI1PK', 'AttributeType': 'S'},
   {'AttributeName': 'GSI1SK', 'AttributeType': 'S'}],
  'TableName': 'chat',
  'KeySchema': [{'AttributeName': 'PK', 'KeyType': 'HASH'},
   {'AttributeName': 'SK', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2023, 3, 17, 12, 29, 29, 944000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/chat',
  'GlobalSecondaryIndexes': [{'IndexName': 'GSI1',
    'KeySchema': [{'AttributeName': 'GSI1PK', 'KeyType

## Helpers

To setup some other keys later on, we create a function to extract the id from a key:

In [6]:
def parse_id_from_key(key):
    return key.split("#")[-1]

parse_id_from_key("CHAN#7")

'7'

## Users

We create and save our users:

In [17]:
user_alice = {
    'PK': { 'S': "USER#12" },
    'SK': { 'S': "USER#12" },
    'Name': { 'S': 'Alice' },
    'Mail': { 'S': 'alice@mail.com' }
}

user_bob = {
    'PK': { 'S': "USER#18" },
    'SK': { 'S': "USER#18" },
    'Name': { 'S': 'Bob' },
    'Mail': { 'S': 'bob@mail.com' }
}

user_eve = {
    'PK': { 'S': "USER#44" },
    'SK': { 'S': "USER#44" },
    'Name': { 'S': 'Eve' },
    'Mail': { 'S': 'eve@mail.com' }
}

def save_user(user):
    return client.put_item(
        TableName="chat",
        Item=user
    )

# AP6
def find_user(key):
    item = client.get_item(
        TableName="chat",
        Key={
          'PK': { 'S': key },
          'SK': { 'S': key }
        }
    )
    return item['Item'] if 'Item' in item else False

save_user(user_alice)
find_user(user_alice['PK']['S'])

{'SK': {'S': 'USER#12'},
 'PK': {'S': 'USER#12'},
 'Mail': {'S': 'alice@mail.com'},
 'Name': {'S': 'Alice'}}

## Channel

We create our channel and functions to save, find and count users and messages in the channel:

In [8]:
channel_town_hall = {
    'PK': { 'S': "CHAN#7" },
    'SK': { 'S': "CHAN#7" },
    'Name': { 'S': 'Town Hall' },
    'Desc': { 'S': 'General News' },
    'UserCount': { 'N': "0" },
    'MessageCount': { 'N': "0" }
}

def save_channel(channel):
    return client.put_item(
        TableName="chat",
        Item=channel
    )

# AP2
def find_channel(key):
    item = client.get_item(
        TableName="chat",
        Key={
          'PK': { 'S': key },
          'SK': { 'S': key }
        }
    )
    return item['Item'] if 'Item' in item else False

# AP4
def message_count_for_channel(channel):
    return int(channel['MessageCount']['N'])

# AP7
def user_count_for_channel(channel):
    return int(channel['UserCount']['N'])

save_channel(channel_town_hall)
channel_town_hall = find_channel(channel_town_hall['PK']['S'])
channel_town_hall

{'SK': {'S': 'CHAN#7'},
 'Desc': {'S': 'General News'},
 'PK': {'S': 'CHAN#7'},
 'UserCount': {'N': '0'},
 'MessageCount': {'N': '0'},
 'Name': {'S': 'Town Hall'}}

In [9]:
message_count_for_channel(channel_town_hall)

0

In [10]:
user_count_for_channel(channel_town_hall)

0

## User Join

When a user joins a channel, we create a new item and increment the user counter. Further, we setup queries to retreive the users for a channel and channels of a user:

In [11]:
def join_channel(user, channel):
    # create userj record
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': "USERJ#" + parse_id_from_key(user['PK']['S']) },
            'Name': user['Name'],
            'JoinedAt': { 'S': str(date.today()) },
            'GSI1PK': user['PK'],
            'GSI1SK': channel['PK'],
            'ChanName': channel['Name']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD UserCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP5
def users_in_channel(key):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :userj)',
        ExpressionAttributeValues={
            ':pk': key,
            ':userj': { 'S': 'USERJ#' }
        }
    )
    return item['Items'] if 'Items' in item else []

# AP1
def channels_for_user(key):
    item = client.query(
        TableName="chat",
        IndexName="GSI1",
        KeyConditionExpression='GSI1PK = :pk AND begins_with(GSI1SK, :chan)',
        ExpressionAttributeValues={
            ':pk': key,
            ':chan': { 'S': 'CHAN#' }
        }
    )
    return item['Items'] if 'Items' in item else []

join_channel(user_alice, channel_town_hall)
join_channel(user_bob, channel_town_hall)
users_in_channel(channel_town_hall['PK'])

[{'ChanName': {'S': 'Town Hall'},
  'GSI1PK': {'S': 'USER#12'},
  'GSI1SK': {'S': 'CHAN#7'},
  'SK': {'S': 'USERJ#12'},
  'JoinedAt': {'S': '2023-03-17'},
  'PK': {'S': 'CHAN#7'},
  'Name': {'S': 'Alice'}},
 {'ChanName': {'S': 'Town Hall'},
  'GSI1PK': {'S': 'USER#18'},
  'GSI1SK': {'S': 'CHAN#7'},
  'SK': {'S': 'USERJ#18'},
  'JoinedAt': {'S': '2023-03-17'},
  'PK': {'S': 'CHAN#7'},
  'Name': {'S': 'Bob'}}]

In [12]:
channels_for_user(user_alice['PK'])

[{'SK': {'S': 'USERJ#12'},
  'JoinedAt': {'S': '2023-03-17'},
  'ChanName': {'S': 'Town Hall'},
  'PK': {'S': 'CHAN#7'},
  'GSI1PK': {'S': 'USER#12'},
  'GSI1SK': {'S': 'CHAN#7'}}]

In [13]:
find_channel(channel_town_hall['PK']['S'])

{'SK': {'S': 'CHAN#7'},
 'Desc': {'S': 'General News'},
 'PK': {'S': 'CHAN#7'},
 'UserCount': {'N': '2'},
 'MessageCount': {'N': '0'},
 'Name': {'S': 'Town Hall'}}

## Messages

When we send a message, we create a new item and increment the messages counter:

In [14]:
def send_message(user, channel, text):
    timestamp = int(time.time())
    message_id = uuid.uuid4()
    
    # create message
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': f"MSG#{timestamp}#{message_id}" },
            'Msg': { 'S': text },
            'CreatedAt': { 'N': str(timestamp) },
            'UserName': user['Name'],
            'UserId': user['PK']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD MessageCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP3
def messages_in_channel(key, limit = 50):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':pk': key,
            ':msg': { 'S': 'MSG#' }
        },
        Limit = limit
    )
    return item['Items'] if 'Items' in item else []
    

send_message(user_bob, channel_town_hall, "Hey there!")
send_message(user_alice, channel_town_hall, "Hello Bob!")
send_message(user_bob, channel_town_hall, "How are you?")
messages_in_channel(channel_town_hall['PK'])

[{'SK': {'S': 'MSG#1679056170#06262520-58ef-4d34-9d4c-0e6197a9615e'},
  'Msg': {'S': 'Hey there!'},
  'PK': {'S': 'CHAN#7'},
  'UserName': {'S': 'Bob'},
  'UserId': {'S': 'USER#18'},
  'CreatedAt': {'N': '1679056170'}},
 {'SK': {'S': 'MSG#1679056170#25f05526-f68b-43c8-bdde-010561bc36ea'},
  'Msg': {'S': 'Hello Bob!'},
  'PK': {'S': 'CHAN#7'},
  'UserName': {'S': 'Alice'},
  'UserId': {'S': 'USER#12'},
  'CreatedAt': {'N': '1679056170'}},
 {'SK': {'S': 'MSG#1679056170#f3510b33-a677-4e4d-8842-62d59f24a246'},
  'Msg': {'S': 'How are you?'},
  'PK': {'S': 'CHAN#7'},
  'UserName': {'S': 'Bob'},
  'UserId': {'S': 'USER#18'},
  'CreatedAt': {'N': '1679056170'}}]

In [15]:
find_channel(channel_town_hall['PK']['S'])

{'SK': {'S': 'CHAN#7'},
 'Desc': {'S': 'General News'},
 'PK': {'S': 'CHAN#7'},
 'UserCount': {'N': '2'},
 'MessageCount': {'N': '3'},
 'Name': {'S': 'Town Hall'}}